In [1]:
import os

import pandas as pd
import numpy as np
import torch

import bitsandbytes
import accelerate
from datasets import Dataset

from modelscope import snapshot_download
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from peft import PeftModel


/root/miniconda3/envs/trans/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-04 21:05:08,318 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-04-04 21:05:08,322 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-04-04 21:05:08,368 - modelscope - INFO - Loading done! Current index file version is 1.13.1, with md5 b5a2c5fe01f7460b3e700a8ce7e6fc94 and a total number of 972 components indexed


In [2]:
model_path=r'/root/autodl-tmp/01ai/Yi-6B-Chat'
model_dir = snapshot_download('01ai/Yi-6B-Chat', cache_dir='/root/autodl-tmp', revision='master')

# base_tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True, padding_side='left', max_length=512, return_tensors='pt')

# base_model = AutoModelForCausalLM.from_pretrained(
# 	 model_path,
# 	 device_map='auto',
# 	 torch_dtype=torch.bfloat16,
# 	 trust_remote_code=True 
# ).eval()

In [3]:
messages = [
    {"role": "user", "content": "Translate the following words into English:\n你係邊個？"},
]

messages_plain = [
    """
    <|im_start|> user
    hi<|im_end|> 
    <|im_start|> assistant
    """
]



# input_ids = base_tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
# output_ids = base_model.generate(input_ids.to('cuda'))
# # response = base_tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True, max_length=100)
# response = base_tokenizer.decode(output_ids[0], skip_special_tokens=False, max_length=100)

# # Model response: "Hello! How can I assist you today?"
# print(response)

In [4]:
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/01ai/Yi-6B-Chat', use_fast=True, padding_side='left', max_length=512, return_tensors='pt')
base_tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/01ai/Yi-6B-Chat', use_fast=True, padding_side='left', max_length=512, return_tensors='pt')
print(len(tokenizer.vocab))
# tokenizer = AutoTokenizer.from_pretrained('/root/AIST4010-Cantonese-Translator/tokenizer', use_fast=True, padding_side='left', max_length=512, return_tensors='pt')

64002


In [5]:
# model = PeftModel.from_pretrained(
#     model,
#    '/root/peft_model',
#     is_trainable=False
# )

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
base_model = AutoModelForCausalLM.from_pretrained(
	 '/root/autodl-tmp/01ai/Yi-6B-Chat',
	 device_map=device,
	 torch_dtype=torch.bfloat16,
     quantization_config=BitsAndBytesConfig(load_in_8bit=True),
	 trust_remote_code=True 
).eval()
model = AutoModelForCausalLM.from_pretrained(
	 model_path,
	 device_map=device,
	 torch_dtype=torch.bfloat16,
	 quantization_config=BitsAndBytesConfig(load_in_8bit=True),
	 trust_remote_code=True 
).eval()
model.resize_token_embeddings(len(tokenizer))
model.load_adapter('/root/autodl-tmp/peft_model_pretrained')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.67s/it]


In [7]:
REPO_DIRECTORY = r'/root/'
ABC_DICT_PATH = r'autodl-tmp/AIST4010-Cantonese-Translator-Data/ABC-Dict/abc_dict.csv'

def load_abc_dataset():
    abc_dict = pd.read_csv(REPO_DIRECTORY + ABC_DICT_PATH)
    abc_dataset = Dataset.from_pandas(abc_dict)
    return abc_dataset

abc_set = load_abc_dataset()
abc_shuffled_set = abc_set.train_test_split(seed=42, test_size=0.1)
abc_train_set = abc_shuffled_set['train']
abc_test_set = abc_shuffled_set['test']

In [8]:
def get_translate_prompt(source_language, target_language, source_text):
    return f"Translate the following words from {source_language} to {target_language}:\n{source_text}"

def get_messages(sample):
    def get_translate_prompt(source_language, target_language, source_text):
        return f"Translate the following words from {source_language} to {target_language}:\n{source_text}"
    return [
        [{"role": "user", "content": get_translate_prompt('Cantonese', 'English', sample['yue'])}],
        [{"role": "user", "content": get_translate_prompt('English', 'Cantonese', sample['en'])}]
    ]

train_samples = abc_train_set.shuffle(seed=10).select(range(20))
test_samples = abc_test_set.shuffle(seed=10).select(range(20))

get_messages(train_samples[0])

[[{'role': 'user',
   'content': 'Translate the following words from Cantonese to English:\n今日我同學唔小心，踢中我左面粒蛋蛋，好痛啊！'}],
 [{'role': 'user',
   'content': "Translate the following words from English to Cantonese:\nToday my classmate wasn't careful, and kicked my left nut. It was very painful!"}]]

In [9]:
def model_output(model, tokenizer, messages, name=None):
        for prompt in messages:
                input_ids = tokenizer.apply_chat_template(conversation=prompt, tokenize=True, add_generation_prompt=True, return_tensors='pt')
                with torch.cuda.amp.autocast():
                        output_ids = model.generate(input_ids.to('cuda'), max_new_tokens=100)
                response = base_tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True, max_length=100)
                # response = tokenizer.decode(output_ids[0], skip_special_tokens=False, max_length=100)
                print(output_ids)
                print(f"{name}:\n{response}\n")


print([model_output(base_model, base_tokenizer, get_messages(train_samples[0]), 'Base model'),])

/root/miniconda3/envs/trans/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/root/miniconda3/envs/trans/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


tensor([[    6,  2942,   144,  7759, 14429,   567,  1926,  3151,   742, 26212,
          2823, 59569,   592,  4750, 59601,   144, 10721, 59646, 54274,   534,
           453,   453, 13272,   101, 62328, 59642, 59646, 60488, 59724, 61329,
         60751, 60751,   101, 59706, 60544, 60530,   103,     7, 59568,   144,
             6, 14135,   144, 25585,   826,  1225, 13580, 24870, 19591,   795,
           594,   567,  1999,  1682,  1935,    97,   648, 26288,   810,  1272,
            99,     7]], device='cuda:0')
Base model:
Today my classmate accidentally kicked me in the left testicle, it hurts so much!

tensor([[    6,  2942,   144,  7759, 14429,   567,  1926,  3151,   742,  4750,
           592, 26212,  2823, 59569, 59601,   144, 25585,   826,  1225, 13580,
          3613, 59610, 59570, 11369,    97,   597, 19591,   826,  1999,  6230,
            98,   983,   717,  1196, 16465,    99,     7, 59568,   144,     6,
         14135,   144,  3569, 59646, 54274, 12988, 13272,   101, 62328, 5

In [10]:
def compare_outputs(samples):
    for sample in samples:
        print(sample)
        prompts = get_messages(sample)
        for prompt in prompts:
            print(f"Prompt:\n{prompt[0]['content']}")
            print()
            model_output(base_model, base_tokenizer, [prompt], 'Base model')
            model_output(model, tokenizer, [prompt], 'Fine-tuned model')

# compare_outputs(train_samples)

In [11]:
model_output(model, tokenizer, get_messages(train_samples[0]), 'Fine-tuned model')

tensor([[    6,  2942,   144,  7759, 14429,   567,  1926,  3151,   742, 26212,
          2823, 59569,   592,  4750, 59601,   144, 10721, 59646, 54274,   534,
           453,   453, 13272,   101, 62328, 59642, 59646, 60488, 59724, 61329,
         60751, 60751,   101, 59706, 60544, 60530,   103,     7, 59568,   144,
             6, 14135,   144, 25585,   826,  1225, 13580, 24870, 19591,   795,
           594,   567,  1999,  1682,  1935,    97,   648,  1403, 26288,    99,
             7]], device='cuda:0')
Fine-tuned model:
Today my classmate accidentally kicked me in the left testicle, it really hurts!

tensor([[    6,  2942,   144,  7759, 14429,   567,  1926,  3151,   742,  4750,
           592, 26212,  2823, 59569, 59601,   144, 25585,   826,  1225, 13580,
          3613, 59610, 59570, 11369,    97,   597, 19591,   826,  1999,  6230,
            98,   983,   717,  1196, 16465,    99,     7, 59568,   144,     6,
         14135,   144, 10721, 59646, 60588, 54274,   534,   453,   453, 132

In [12]:
compare_outputs(train_samples)


{'en': "Today my classmate wasn't careful, and kicked my left nut. It was very painful!", 'yue': '今日我同學唔小心，踢中我左面粒蛋蛋，好痛啊！'}
Prompt:
Translate the following words from Cantonese to English:
今日我同學唔小心，踢中我左面粒蛋蛋，好痛啊！



tensor([[    6,  2942,   144,  7759, 14429,   567,  1926,  3151,   742, 26212,
          2823, 59569,   592,  4750, 59601,   144, 10721, 59646, 54274,   534,
           453,   453, 13272,   101, 62328, 59642, 59646, 60488, 59724, 61329,
         60751, 60751,   101, 59706, 60544, 60530,   103,     7, 59568,   144,
             6, 14135,   144, 25585,   826,  1225, 13580, 24870, 19591,   795,
           594,   567,  1999,  1682,  1935,    97,   648, 26288,   810,  1272,
            99,     7]], device='cuda:0')
Base model:
Today my classmate accidentally kicked me in the left testicle, it hurts so much!

tensor([[    6,  2942,   144,  7759, 14429,   567,  1926,  3151,   742, 26212,
          2823, 59569,   592,  4750, 59601,   144, 10721, 59646, 54274,   534,
           453,   453, 13272,   101, 62328, 59642, 59646, 60488, 59724, 61329,
         60751, 60751,   101, 59706, 60544, 60530,   103,     7, 59568,   144,
             6, 14135,   144, 25585,   826,  1225, 13580, 24870, 19591,  

In [13]:
compare_outputs(test_samples)

{'en': 'Miss Linda.', 'yue': 'Linda姐。'}
Prompt:
Translate the following words from Cantonese to English:
Linda姐。



tensor([[    6,  2942,   144,  7759, 14429,   567,  1926,  3151,   742, 26212,
          2823, 59569,   592,  4750, 59601,   144, 59620, 27164, 60706,   102,
             7, 59568,   144,     6, 14135,   144, 59620, 27164, 60706,    98,
             7]], device='cuda:0')
Base model:
Linda姐.

tensor([[    6,  2942,   144,  7759, 14429,   567,  1926,  3151,   742, 26212,
          2823, 59569,   592,  4750, 59601,   144, 59620, 27164, 60706,   102,
             7, 59568,   144,     6, 14135,   144, 59620, 27164, 60706,   101,
         62376, 59967, 12666, 60521, 30916, 59568,   534,   457,   438, 11940,
           102,     7]], device='cuda:0')
Fine-tuned model:
Linda姐，係指一個叫 Linda 嘅女士。

Prompt:
Translate the following words from English to Cantonese:
Miss Linda.

tensor([[    6,  2942,   144,  7759, 14429,   567,  1926,  3151,   742,  4750,
           592, 26212,  2823, 59569, 59601,   144, 30874, 30916,    98,     7,
         59568,   144,     6, 14135,   144, 30874, 30916,    98,     7

In [ ]:
messages = [
    {"role": "user", "content": "Translate the following words into English:\n乜嘢都係波士決定嘅，打工仔啲人淨係得個知字。\n"},
]

# get 5 random samples from train and test dataset
train_sample = abc_train_set.shuffle(seed=42).select(range(5))
test_sample = abc_test_set.shuffle(seed=42).select(range(5))

en_train_messages = {get_translate_prompt('Cantonese', sentence) for sentence in train_sample['en']}
en_test_messages = {get_translate_prompt('Cantonese', sentence) for sentence in test_sample['en']}
yue_train_messages = {get_translate_prompt('English', sentence) for sentence in train_sample['yue']}
yue_test_messages = {get_translate_prompt('English', sentence) for sentence in test_sample['yue']}

for messages in [en_train_messages, en_test_messages, yue_train_messages, yue_test_messages]:
    for message in messages:
        print(message)


TypeError: get_translate_prompt() missing 1 required positional argument: 'source_text'

In [ ]:
input_ids = tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
output_ids = model.generate(input_ids.to('cuda'))
# response = base_tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True, max_length=100)
response = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True, max_length=100)

# Model response: "Hello! How can I assist you today?"
print("Tuned model:", response)

input_ids = base_tokenizer.apply_chat_template(conversation=messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
output_ids = base_model.generate(input_ids.to('cuda'))
# response = base_tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True, max_length=100)
response = base_tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True, max_length=100)

print("Base model:", response)

UndefinedError: 'str object' has no attribute 'role'